# FMA: A Dataset For Music Analysis

Kirell Benzi, Michaël Defferrard, Pierre Vandergheynst, Xavier Bresson, EPFL LTS2.

## Usage

1. Download dataset from <https://github.com/mdeff/fma>.
2. Uncompress the archive, e.g. with `unzip fma_small.zip`.
3. Load and play with the data in this notebook.

In [ ]:
%matplotlib inline

import utils
import librosa
from sklearn.preprocessing import StandardScaler
from sklearn.svm import SVC
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import IPython.display as ipd
import os.path

In [ ]:
DATA_DIR = os.environ.get('DATA_DIR')
df = pd.read_json(os.path.join(DATA_DIR, 'fma_small.json'))
path = utils.build_path(df, DATA_DIR)

## 1 Metadata

The metadata table, a JSON file in the root directory of the archive, is composed of many colums:
1. The index is the ID of the song, taken from the FMA, used as the name of the audio file.
2. Meta-data from the Free Music Archive website (4 columns).
3. A column to indicate if the song is part of the training or testing set.
4. For the `small` and `medium` datasets, data colected from the [Echonest](http://the.echonest.com/) API:
    1. Meta-data (5 columns).
    2. Temporal features: one vector per song.
    3. Audio features (8 columns).
    4. Social features (5 columns).
    5. Rankings (5 columns).

In [ ]:
ipd.display(df[utils.FMA_METADATA].head())
ipd.display(df[utils.ECHONEST_METADATA].head())
ipd.display(df[utils.ECHONEST_AUDIO_FEATURES].head())
ipd.display(df[utils.ECHONEST_SOCIAL_FEATURES].head())
#ipd.display(df[utils.ECHONEST_RANKS].head())

In [ ]:
x = df.loc[10060, utils.ECHONEST_TEMPORAL_FEATURES]
plt.figure(figsize=(15, 5))
plt.plot(x);

# 2 Audio

You can listen to an audio excerpt with the below code.

In [ ]:
filename = path(0)
print('File: {}'.format(filename))
ipd.Audio(filename)

And use [librosa](https://github.com/librosa/librosa) to extract the raw waveform and compute audio features.

In [ ]:
x, sr = librosa.load(filename)
print('Duration: {:.2f}s, {} samples'.format(x.shape[0] / sr, x.size))
ipd.display(ipd.Audio(data=x, rate=sr))

plt.figure(figsize=(15, 5))
plt.plot(x)

plt.figure(figsize=(15, 5))
S, freqs, bins, im = plt.specgram(x, NFFT=1024, Fs=sr, noverlap=512)

## 3 Genre classification

### 3.1 From Echonest features

In [ ]:
# Select features.
features = utils.ECHONEST_AUDIO_FEATURES + utils.ECHONEST_SOCIAL_FEATURES

# Discard songs with NaN Echonest features.
# TODO: fix dataset.
keep = df[features].isnull().apply(lambda x: not x.any(), axis=1)
df = df[keep]

X = df[features].as_matrix()
y = df.top_genre.as_matrix()

In [ ]:
train = df['train'] == True

y_train = y[train]
y_test = y[~train]
X_train = X[train]
X_test = X[~train]

print('{} training examples, {} testing examples'.format(y_train.size, y_test.size))
print('{} features, {} classes'.format(X_train.shape[1], np.unique(y_train).size))

In [ ]:
# Standardize features by removing the mean and scaling to unit variance.
scaler = StandardScaler(copy=False)
scaler.fit_transform(X_train)
scaler.transform(X_test)

# Support vector classification.
clf = SVC()
clf.fit(X_train, y_train)
score = clf.score(X_test, y_test)
print('Accuracy: {:.2f} %'.format(score*100))

### 3.2 From raw audio

### 3.3 From hand-crafted audio features